# Main Notebook

The model was trained in "training_model.ipynb" file in the same folder

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib --user

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Add Mediapipe model - Holistic

In [4]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    
    # Colour Detection - START 
    
    color_str = []
   
    try:
        for coords in coords_arr:
            #Need to check whether it gets the real colors from the video
            coords_revert = coords[::-1]
            color = image[coords_revert]
        #     color = image[123,903]
        #     color = image[coords_shoulder]
            r = color[2]
            g = color[1]
            b = color[0]


            color_str.append(recognize_color(r,g,b))
    except:
        pass
    # Colour Detection - END
    
    return image, results, color_str

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [7]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

#  Action Detection - Model uploaded 

In [8]:
# Defining the actions array for the different actions recognized

# actions = np.array(['Point', 'Penalty', 'Ignore'])

actions = np.array(['Point', 'Penalty', 'Ignore'])

In [9]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [14]:
del model

In [15]:
# Creating the Model - It needs to be the same model structure / architecture as the one that needs to be loaded

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

#### Load Different Models 

In [16]:
# If the model is already trained then it only needs to be loaded 
# check actionDetection folder for the training

model.load_weights('actionDetection_3.h5')
# model.load_weights('actionDetection_4Classes_01.h5')
# model.load_weights('actionDetection_50P.h5')

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 64)            82688     
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 99        
Total params: 237,251
Trainable params: 237,251
Non-trainable params: 0
________________________________________________

# Differentiation between a referee and a player 

In [18]:
import pandas as pd

index_blue=["color_name", "R", "G", "B"]
csv_blue = pd.read_csv('colors_blue.csv', delimiter=";", names=index_blue, header=None)

In [19]:
print(csv_blue)

       color_name    R    G    B
0           Black    0    0    0
1    Darkest Blue    8   22   39
2            Navy   17   46   81
3       Dark Blue   32   84  147
4            Blue   46  120  210
5      Light Blue  109  161  224
6    Lighter Blue  151  188  233
7   Lightest Blue  193  215  242
8            Grey  120  144  156
9      Light Grey  167  192  205
10   Lighter Grey  200  215  223
11  Lightest Grey  232  293  242


In [20]:
#  Could be either .all() or any()

# If we want to recognize only Blue then we need a different color dataset 
# If we want to recognize only White then we only need the sum of all r,g,b values to be non-less then 690

# Calculating the distance from the actual colors from the dataset 
# and whichever has the lowest distance is assigned 
def recognize_color(r,g,b):
 
    minimum = 10000 
    cname = ""
    for i in range(len(csv_blue)):
        distance = abs(r - int(csv_blue.loc[i,"R"])) + abs(g - int(csv_blue.loc[i,"G"])) + abs(b - int(csv_blue.loc[i,"B"]))
        if (distance <= minimum).all():
            minimum = distance
            cname = csv_blue.loc[i,"color_name"]
    return cname

In [21]:
def color_detection(image, landmarks, color_str):
    coords_shoulder = tuple(np.multiply(
            np.array(
                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x, 
                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y))
        , [1280,720]).astype(int))

    coords_right_hip = tuple(np.multiply(
            np.array(
                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].x, 
                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y))
        , [1280,720]).astype(int))

    coords_left_hip = tuple(np.multiply(
            np.array(
                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].x, 
                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y))
        , [1280,720]).astype(int))

    coords_arr = [coords_shoulder, coords_right_hip, coords_left_hip]


    if "Navy" in color_str or "Darkest Blue" in color_str or "Dark Blue" in color_str:

        cv2.rectangle(image, (590, 570), (800, 610), (0, 255, 0), -1)
        cv2.putText(image, "A Referee", (600, 600), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    else:
        cv2.rectangle(image, (590, 570), (800, 610), (0, 0, 255), -1)
        cv2.putText(image, "A Player", (600, 600), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.putText(image, "S - {}, LH - {}, RH - {}".format(color_str[0], color_str[1], color_str[2]), (300, 700), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    

In [22]:
# A method for combining all three colours into one number that can be compared. 

def distinguish_referee(color_str):
    
    amount = 0 
    
    for color in color_str:
        if color == "Black":
            amount+= 100
        if color == "Darkest Blue":
            amount+=90
        if color == "Navy":
            amount+=80
        if color == "Dark Blue":
            amount+=70

        if color == "Grey":
            amount+=40
        if color == "Light Grey":
            amount+=30
        if color == "Lighter Grey":
            amount+=20
        if color == "Lightest Grey":
            amount+=10
            
    return amount
        
        
    

# Calculating & Visualising Anlges  

Angles of the elbows (between hand and shoulder) could also be calcuated. This would help to differentiate between a point and a penalty from category 2

In [23]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [24]:
def get_angle(landmarks, image):
    # Get coordinates for Left Hand
    hip = [landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].y]
    shoulder = [landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].y]
    elbow = [landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].y]

    # Get coordinates for Right Hand
    hip_right = [landmarks[mp_holistic.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_HIP.value].y]
    shoulder_right = [landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].y]
    elbow_right = [landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].y]
    
    # Calculate angles
    angle = calculate_angle(hip, shoulder, elbow)
    angle_right = calculate_angle(hip_right, shoulder_right, elbow_right)
    
    #Position of displaying the angle for the Left Side
    angle_position_list = list(np.multiply(shoulder, [1280, 720]).astype(int))
    angle_position_list[0] = angle_position_list[0]+30 #Placing it 30px on the right
    angle_position_tuple = tuple(angle_position_list)

    #Position of displaying the angle for the Right Side
    angle_right_position_list = list(np.multiply(shoulder_right, [1280, 720]).astype(int))
    angle_right_position_list[0] = angle_right_position_list[0]-30 #Placing it 30px on the left
    angle_right_position_tuple = tuple(angle_right_position_list)

    # Visualize angle
    cv2.putText(image, str(angle.astype(int)) + " C", 
                   angle_position_tuple, 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.putText(image, str(angle_right.astype(int)) + " C", 
                   angle_right_position_tuple, 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    
    return angle, angle_right
    

# Points Deduction 

In [25]:
def points_deduction(angle, angle_right, points):
    # For the Left Hand 
    if angle >= 20 and angle < 60:
        points = "1_P Left"

    elif angle >= 60 and angle < 110:
        points = "2_P Left"

    elif angle >= 110:
        points = "3_P Left"

    #For the Right Hand 
    if angle_right >= 20 and angle_right < 60:
        points = "1_P Right"

    elif angle_right >= 60 and angle_right < 110:
        points = "2_P Right"

    elif angle_right >= 110:
        points = "3_P Right"
        
    return points

# Scoreboard 


In [26]:
#the size of the scoreboard without the penalty is with 50 lower (in height)

def scoreboard_design(image, count_left, count_right):

    cv2.rectangle(image, (1040,620), (1150, 720), (255, 0, 0), -1)
    cv2.rectangle(image, (1150, 620), (1260, 720), (0, 0, 255), -1)
    cv2.rectangle(image, (1115, 600), (1185, 620), (255, 255, 255), -1)
    
    cv2.putText(image, "Score", 
           (1130,614), 
           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        
    cv2.putText(image, str(count_left), 
           (1090,650), 
           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    
    cv2.putText(image, str(count_right), 
           (1200,650), 
           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    
    cv2.line(image, (1040, 670), (1260, 670), (255, 255, 255), 2, cv2.LINE_AA)

In [ ]:
#     if (threes == 1 and predictions[-controlSize:][0] == 3) or (twos == 1 and predictions[-controlSize:][0] == 2):

In [27]:
def scoreboard_calc(points, countPoints_left, countPoints_right, controlSize, predictions):
    zeros = 0
    ones = 0
    twos = 0
    threes = 0
    
    # controlSize = 10 by default

    for i in range(controlSize):
        if predictions[-controlSize:][i] == 0:
            zeros += 1
        elif predictions[-controlSize:][i] == 1:
            ones += 1
        elif predictions[-controlSize:][i] == 2:
            twos += 1
        elif predictions[-controlSize:][i] == 3:
            threes += 1

    # It needs to check if the ignore is only 1 and then if this ignore is in the first position 
    # if it doesn't check the number of ignores in this list it will enter the if as many times as 
    # it has ignores
    
    # If we want to change to look no at the "Ignore", but some other signal - change both values 
    # in the if statement

    if ones == 1 and predictions[-controlSize:][0] == 1:
        if points == "1_P Left":
            countPoints_left += 1
        elif points == "2_P Left": 
            countPoints_left += 2
        elif points == "3_P Left": 
            countPoints_left += 3
        elif points == "1_P Right":
            countPoints_right += 1
        elif points == "2_P Right": 
            countPoints_right += 2
        elif points == "3_P Right": 
            countPoints_right += 3
    
    return countPoints_left, countPoints_right  

### Rendering & Visualization of the Model  

In [30]:
from scipy import stats

In [31]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
# colors = [(245,117,16), (117,245,16), (255, 0, 137), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [32]:
video_Karate_Action_Points = "France_Spain_2010_Action_Points.mp4"
video_Karate_24 = "KarateProject_Todor_24_No_Detection.mp4"

In [33]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.7

# Points recognized based on the angles
points = ""
penalty = ""

#Scoreboard
countPoints_left = 0
countPoints_right = 0
frameCounter_fps = 0
FrameCounter = 0
controlSize = 6 #This is really important - it should differ for different output

# Colour Detection
coords_arr = []
color_str = []
color_amount = 0

cap = cv2.VideoCapture(0) # Set the "0" value to "video_Karate_Action_Points" to try with a video
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    
    #this depends on the output video that is going to be put | the video and the internal camera need to be with the same size
    cap.set(3, 1280)
    cap.set(4, 720)
    
    print(cap.get(3))
    print(cap.get(4))
    print(cap.get(cv2.CAP_PROP_FPS))
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        

        # Make detections
        image, results, color_str = mediapipe_detection(frame, holistic)

        #This code is only for Legend videos - to focus the mediapipe model in the centre - to the referee
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
#         results = holistic.process(image[50:500, 310:800])                 
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


#         print(results)
        
        # Calculate the angles between the hip / arm 
        try:
            
            landmarks = results.pose_landmarks.landmark
            
            angle, angle_right = get_angle(landmarks, image) #Method for calculating the angles and visualising them
            
        
        except:
            pass

        # Draw landmarks 
        draw_styled_landmarks(image, results)
        
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))


        #3. Viz logic
            if np.unique(predictions[-5:])[0]==np.argmax(res): # This code is holding the a specific prediction for n frames before it confirms that it is trully the one that we want (better for transitioning between classes)
                if res[np.argmax(res)] > threshold:

                    # START - Points Deduction

                    if actions[np.argmax(res)] == "Point": 

                        #A method for deducting the points received
                        points = points_deduction(angle, angle_right, points) 

                        #START - Scoring Points

                        try:
                        # A method for updating the scoreboard based on the points received
                            countPoints_left, countPoints_right = scoreboard_calc(points, countPoints_left, countPoints_right, controlSize, predictions)

                        except:
                            pass
                        #END - Scoring Points

                    else:
                        points = ""

                    # END - Points Deduction

# #             Viz probabilities
#             image = prob_viz(res, actions, image, colors)
        try:

            if points != "":
                cv2.rectangle(image, (1042,580), (1252, 540), (255, 117, 16), -1)
                cv2.putText(image, points, (1052,570), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            # Check this 
            if actions[np.argmax(res)] == "Penalty":
                cv2.rectangle(image, (1042,580), (1252, 540), (16,117,245), -1)
                cv2.putText(image, "Ignore", (1052,570), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
            cv2.putText(image, "Predictions:", (1052,530), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass
       
    # color detection
        
        try:
            
#             color_detection(image, landmarks, color_str)

            coords_shoulder = tuple(np.multiply(
                    np.array(
                        (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x, 
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y))
                , [1280,720]).astype(int))

            coords_right_hip = tuple(np.multiply(
                    np.array(
                        (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].x, 
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y))
                , [1280,720]).astype(int))

            coords_left_hip = tuple(np.multiply(
                    np.array(
                        (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].x, 
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y))
                , [1280,720]).astype(int))

            coords_arr = [coords_shoulder, coords_right_hip, coords_left_hip]
            
            color_amount = distinguish_referee(color_str)


            if color_amount > 170:

                cv2.rectangle(image, (590, 670), (800, 710), (0, 255, 0), -1)
                cv2.putText(image, "A Referee", (600, 700), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                
 
            else:
                cv2.rectangle(image, (590, 670), (800, 710), (0, 0, 255), -1)
                cv2.putText(image, "A Player", (600, 700), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

#             cv2.putText(image, "S - {}, LH - {}, RH - {}".format(color_str[0], color_str[1], color_str[2]), (300, 700), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
        except:
            pass
  
                
#         # Scoreboard - Visualisation
        scoreboard_design(image, countPoints_left, countPoints_right)
        
#         # Frame Counter & Current FPS
#         frameCounter_fps += 1
#         cv2.putText(image, str(frameCounter_fps), (1050, 550), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
#         if len(predictions) > 14: 
#             cv2.putText(image, str(predictions[-14:]), (550, 500), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
#         cv2.putText(image, points, (650, 600), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1280.0
720.0
1280.0
720.0
25.0
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
Penalty
P

In [148]:
cap.release()
cv2.destroyAllWindows()